test code for deep learning archtects

In [1]:
import os
import json
import datetime

import pandas as pd
import numpy as np

import driverlessai
driverlessai.__version__

'1.10.1.3'

In [2]:
# Driverless AIのuser nameとpasswordの読み込み
with open(os.path.join('..', '..', '..', 'idpass.json')) as f:
    idpass = json.load(f)

In [18]:
dai = driverlessai.Client(address='http://35.172.135.60', username=idpass['id'], password=idpass['pass11013gpu'])
dai

<class 'driverlessai._core.Client'> http://35.172.135.60

In [5]:
{i.name:i.key for i in dai.datasets.list()}

{'histopathology_train.zip': '8ec3abf4-a407-11ec-8921-0242ac110002',
 'car_deals.zip': '50119e5c-a407-11ec-8921-0242ac110002',
 'UCI_Credit_Card3.csv': 'ac9c94bc-9f2d-11ec-ad99-0242ac110002',
 'kaggle_train.csv': '056c70f0-9365-11ec-96ca-0242ac110002',
 'talk4_customize_NLP検証用_h連携.csv': '9a306ad0-9364-11ec-96ca-0242ac110002',
 'BostonHousing.csv': '6db78e82-91f9-11ec-a31a-0242ac110002'}

In [6]:
ds = dai.datasets.get('50119e5c-a407-11ec-8921-0242ac110002') 
ds

<class 'Dataset'> 50119e5c-a407-11ec-8921-0242ac110002 car_deals.zip

In [13]:
# Expert設定のパラメータの検索
dai.experiments.search_expert_settings('pretrained')

enable_pytorch_nlp | default_value: auto
pytorch_nlp_pretrained_models | default_value: ['bert-base-uncased', 'distilbert-base-uncased', 'bert-base-multilingual-cased']
tensorflow_nlp_pretrained_embeddings_file_path | default_value: 
tensorflow_nlp_pretrained_s3_access_key_id | default_value: 
tensorflow_nlp_pretrained_s3_secret_access_key | default_value: 
tensorflow_nlp_pretrained_embeddings_trainable | default_value: False
pytorch_nlp_pretrained_models_dir | default_value: 
enable_tensorflow_image | default_value: auto
tensorflow_image_pretrained_models | default_value: ['xception']
tensorflow_image_fine_tune | default_value: False
tensorflow_image_fine_tuning_num_epochs | default_value: 2
tensorflow_image_augmentations | default_value: ['HorizontalFlip']
tensorflow_image_pretrained_models_dir | default_value: ./pretrained/image/


dai.experiments.createのkeyword Argumentsに関して
- https://docs.h2o.ai/driverless-ai/pyclient/docs/html/client.html#driverlessai._experiments.Experiments.create

DLのアーキテクチャ設定（tensorflow_image_pretrained_models）
- https://docs.h2o.ai/driverless-ai/latest-stable/docs/userguide/expert_settings/image_settings.html#tensorflow-image-pretrained-models

In [9]:
df_expperiments_info = pd.read_csv('exp_params_test.csv')
df_expperiments_info

,try_n,data_name,s3url,target_column,task,drop_columns,enable_gpus,test_mode,tensorflow_image_pretrained_models
0,1,car_deals.zip,s3://h2o-public-test-data/bigdata/server/Image...,Price,regression,"Manufacturer,Model,Year,Category,Mileage,FuelT...",NaN,True,efficientnetb0


In [10]:
row = df_expperiments_info.iloc[0,:]

In [11]:
# データセットの学習、テストセットへの分割
ds_split = ds.split_to_train_test(train_size=0.75, train_name=row['data_name'] + '_train', test_name=row['data_name'] + '_test')
ds_split

Complete


{'train_dataset': <class 'Dataset'> a0fd39d0-b3d3-11ec-8aea-0242ac110002 car_deals.zip_train,
 'test_dataset': <class 'Dataset'> a0fd6220-b3d3-11ec-8aea-0242ac110002 car_deals.zip_test}

In [12]:
row = df_expperiments_info.iloc[0,:]

s3url = row['s3url']  # DAIにアップされてない場合の取得先S3
# Experiment設定
target_column = row['target_column']
task = row['task']    # 'regression', 'classification', or 'unsupervised'
if row['drop_columns']  is np.nan:     # dropped clmを指定しない場合
    drop_columns = []
else:
    drop_columns = row['drop_columns'] .split(',')     # strをList化

pretrained = [row['tensorflow_image_pretrained_models']]

dai_settings = {
    #'train_dataset': ds, 
    'target_column': target_column,
    'task': task,
    'drop_columns': drop_columns,
    'tensorflow_image_pretrained_models':[df_expperiments_info['tensorflow_image_pretrained_models'][0]]
}
if row['test_mode']:    # test modeの時、Acc=1&Time=1のExperimentを実施
    dai_settings['accuracy'] = 1
    dai_settings['time'] = 1

dai_settings

{'target_column': 'Price',
 'task': 'regression',
 'drop_columns': ['Manufacturer',
  'Model',
  'Year',
  'Category',
  'Mileage',
  'FuelType',
  'EngineVolume',
  'DriveWheels',
  'GearBox',
  'Doors',
  'Wheel',
  'Color',
  'InteriorColor',
  'LeatherInterior'],
 'tensorflow_image_pretrained_models': ['efficientnetb0'],
 'accuracy': 1,
 'time': 1}

In [116]:
ex = dai.experiments.create(**ds_split, **dai_settings)
ex

Experiment launched at: http://3.95.6.232/#/experiment?key=4a0230f8-b221-11ec-9c59-0242ac110002
Complete 100.00% - Status: Complete                                                


<class 'Experiment'> 4a0230f8-b221-11ec-9c59-0242ac110002 mabanoge

In [123]:
data_name = row['data_name']
log_dataname = data_name
log_try_n = row['try_n']
log_datasize_mb = ds.file_size/1024**2     # 元データサイズ（MB）
log_n_observation = ds.shape[0]    # オブザベーション数
log_n_features = ds.shape[1] - len(drop_columns) - 1    # 特徴量数
log_duration_min = ex.run_duration/60    # 実行時間（分）
log_expsize_mb = ex.size/1024**2     # Experimentサイズ（MB）
log_accuracy_time_interpretability = ex.settings['accuracy'], ex.settings['time'], ex.settings['interpretability']   # accuracy, time, interpretability

print(log_datasize_mb)
print(log_n_observation)
print(log_n_features)
print(log_duration_min)
print(log_expsize_mb)
print(log_accuracy_time_interpretability)

749.219669342041
34571
1
7.225246338049571
1152.5995473861694
(1, 1, 8)


In [126]:
save_dict = dict(Data_Name=data_name,
                 Try=row['try_n'],
                 Datasize_mb = ds.file_size/1024**2,
                 N_Observation = ds.shape[0],
                 N_features = ds.shape[1] - len(drop_columns) - 1,
                 Duration_min = ex.run_duration/60,
                 Experiment_Size_mb = ex.size/1024**2,
                 Acc_Time_Interpret = (ex.settings['accuracy'], ex.settings['time'], ex.settings['interpretability']),
                 Metrics = ex.metrics()
                )
save_dict

{'Data_Name': 'car_deals.zip',
 'Try': 1,
 'Datasize_mb': 749.219669342041,
 'N_Observation': 34571,
 'N_features': 1,
 'Duration_min': 7.225246338049571,
 'Experiment_Size_mb': 1152.5995473861694,
 'Acc_Time_Interpret': (1, 1, 8),
 'Metrics': {'scorer': 'RMSE',
  'val_score': 5675.96630859375,
  'val_score_sd': 139.94525146484375,
  'val_roc_auc': None,
  'val_pr_auc': None,
  'test_score': 5193.61865234375,
  'test_score_sd': 139.94525146484375,
  'test_roc_auc': None,
  'test_pr_auc': None}}

In [132]:
#ds_split['train_dataset'].delete()
#ds_split['test_dataset'].delete()